In [ ]:
import sys
import os

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import urllib
import requests
import csv, pandas, re, numpy

hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

: 

# Testing and playground

In [20]:
kk = '<div class="content">\n<div id="post_message_34705352">\n<blockquote class="postcontent restore">\nVielen Dank, <img alt="" border="0" class="inlineimg" src="https://bfriends.brigitte.de/foren/images/brigitte/smilies/neue_smileys_29.gif" title="Smile"> liebes Community-Team. Es ist so wie eine "Reinigung" <i>des</i> Threads ... vielleicht kommen jetzt<b>neue</b>Fragen, Sorgen, Lösungen, <b>Ideen</b> auf  ... manchmal ist weniger, mehr ... <img alt="" border="0" class="inlineimg" src="https://bfriends.brigitte.de/foren/images/brigitte/smilies/neue_smileys_29.gif" title="Zwinker">\n</blockquote>\n</div>\n</div>'
# print(kk)
s  = BeautifulSoup(kk)
for i in s.find_all('img'):
    t = i.get('title').lower()
    i.replace_with(":"+t+":")
print(s)
s.getText(separator = " ", strip=True)

<div class="content">
<div id="post_message_34705352">
<blockquote class="postcontent restore">
Vielen Dank, :smile: liebes Community-Team. Es ist so wie eine "Reinigung" <i>des</i> Threads ... vielleicht kommen jetzt<b>neue</b>Fragen, Sorgen, Lösungen, <b>Ideen</b> auf  ... manchmal ist weniger, mehr ... :zwinker:
</blockquote>
</div>
</div>


'Vielen Dank, :smile: liebes Community-Team. Es ist so wie eine "Reinigung" des Threads ... vielleicht kommen jetzt neue Fragen, Sorgen, Lösungen, Ideen auf  ... manchmal ist weniger, mehr ... :zwinker:'

# Main functions - actual data

In [4]:
def get_quote_user(x):
    y = x.find('div', class_='bbcode_postedby')
    if (y is not None):
        return(y.find('strong').text)
    else:
        return("None")
    
def get_quote_msg(x):
    y = x.find('div', class_="message")
    z = x.find('div', class_="quote_container")
    if (y is not None):
        return(y.getText(strip = True))
    elif (z is not None):
        return(z.getText(strip = True))
    else:
        return("None")

def process(page_id):
    baseurl = "https://bfriends.brigitte.de/foren/coronavirus/536653-kinder-familie-kita-schule-in-zeiten-von-corona-" + str(page_id) + ".html"
    req = Request(baseurl, data = None, headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)

    usernames = [i.getText() for i in soup.find_all(["a", "span"], {"class": "username"})]
    dates = [i.find(text=True, recursive = False) for i in soup.find_all("span", {"class": "date"})]
    times = [i.find('span', {"class": "time"}).getText() for i in soup.find_all("div", {"class": "posthead"})]

    meta = pandas.DataFrame({'user': usernames, 'date': dates, 'time': times})

    post = []
    
    for i in soup.find_all('div', class_=('content')):
        
        ix_ = i.find(id = re.compile('post_message'))
        # cover cases deleted without post_message id
        ix = None
        if (ix_ is not None):
            ix = ix_.get('id')
    
        # block quoted text
        j = i.find_all('div', class_="bbcode_container") # can quote more than one m
        u, m = None, None
        if j:
            u = [get_quote_user(q) for q in j] # some does not have quoted user!!!
            m = [get_quote_msg(q) for q in j]
            [q.decompose() for q in j] # remove all quotes

        # message replace image emoji with its code (in German)
        for emoji in i.find_all('img'):
            title = emoji.get('title').lower()
            emoji.replace_with(":"+title+":")

        ms = i.getText(separator = " ", strip = True)

        post.append([ix] + [ms] + [u] + [m])
    
    post = pandas.DataFrame(post, columns = ['mid', 'text', 'user_quoted', 'quoted_text'])
    
    return(meta.join(post))

Testing, outputs need to be
- icons image replaced by title text --> to be converted later to UTF-8 code
- bold and italic to have space to avoid words sticking together

In [23]:
process(1).to_csv("/Users/knguyen/Downloads/testi.csv")

# Get data

Error in page257
Error in page288
Error in page449

In [ ]:
for pid in range(279, 1121+1):
    print(pid, end=".")
    op = process(pid)
    try:
        op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + str(pid) + ".csv")
    except:
        print("\nError in page"+str(pid)+"\n")
        pass

In [7]:
# 257 emoji
op = process(257)
op.text[7]

'So kenne ich das aus meiner Kindheit auch. 4 km Rad zum Bahnhof, dann 30 Minuten Bimmelbahn, jeweils einfach einfach. Aber ich bin ja auch hier "eingewandert". Ältere Schüler können ja durchaus morgens mit einem Elternteil in die Kreisstadt fahren, die kriegen, mit Geld versehen, die Zeit schon rum-Mäckes liegt in der Nähe \ud83d\ude02 Aber 10 jährige in der Stadt "rumbutschern" lassen wäre nicht so recht meins.'

In [8]:
op.text[7] = op.text[7].replace('\ud83d\ude02', ":joy:")
print(op.text[7])
op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + '257' + ".csv")

So kenne ich das aus meiner Kindheit auch. 4 km Rad zum Bahnhof, dann 30 Minuten Bimmelbahn, jeweils einfach einfach. Aber ich bin ja auch hier "eingewandert". Ältere Schüler können ja durchaus morgens mit einem Elternteil in die Kreisstadt fahren, die kriegen, mit Geld versehen, die Zeit schon rum-Mäckes liegt in der Nähe :joy: Aber 10 jährige in der Stadt "rumbutschern" lassen wäre nicht so recht meins.


In [9]:
op = process(288)
op.text[3] = op.text[3].replace('\ud83d\ude23', ":persevere:")
op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + '288' + ".csv")
op.text[3]

'In den Schulen meiner Kinder (3. Klasse und 7. klasse Gymnasium) läuft vieles gut und ich habe das Gefühl, dass sich vor allem das Gymnasium auch sehr gewissenhaft auf die Möglichkeit einer Schulschließung vorbereitet. (Sie versuchen zB schon jetzt immer wieder das schul-eigene online Lernportal im kleinen Rahmen zu nutzen um eventuelle Probleme jetzt auszumerzen) Trotzdem kam heute eine E-Mail, dass die Schulkonferenz - entgegengesetzt zur bei uns allgemeingültigen Richtlinie - eine ständige Maskenpflicht beschlossen hat. :knatsch: Die Schulleitung ist offensichtlich sehr besorgt und rechnet quasi täglich mit einem neuen lockdown und ich weiß auch, dass sie viele Lehrer mit Risikofaktoren haben, die größtenteils trotzdem Präsenzunterricht halten. Ich bin auch sehr dankbar, dass es überhaupt wieder regulär Schule gibt. Aber ich finde es für meine (gerade noch) 11-jährige ehrlich gesagt trotzdem ziemlich hart den ganzen Tag (meist von 8-15:15uhr) mit nur kurzen Unterbrechungen in den H

In [ ]:
op = process(499)

: 

In [ ]:
op.text[9]

In [ ]:
op.text[3] = op.text[3].replace('\ud83d\ude23', u'\U0001f623 ')
op.emoji[3] = 'PERSEVERING'
# https://charbase.com/1f623-unicode-persevering-face
op.to_csv("/Users/knguyen/Documents/WIP/Papers/Astrid/data/new/page_" + '288' + ".csv")
op.text[3]